# Kaizen API instructions and examples
***


*This is intended to be a reference document for the BPRC team working within Carleton University. This documents shows several examples API calls and work flows. Excerpts from Coppertree's documentation are included, along with a working python implementations.
<br/><br/>
To get supplementary files including Kaizen API login information and runnable examples, contact Connor Brackley (connor.brackley@mail.concordia.ca)*

**UPDATE**: Since the initial release of this document, Kaizen has updated their API. The relevant updates are now included in this reference document.

### **Table of Contents:**
- [Reference doucments](#Reference-doucments) <br>
- [Notebook setup](#Notebook-setup) <br>
- [Public Trend Log API](#Public-Trend-Log-API) <br>
- [Batch Download Public Trend Log API](#Batch-Download-Public-Trend-Log-API) <br>
- [Access to other API's via the JWT Token](#Access-to-other-API's-via-the-JWT-Token) <br>
- [Get list of trend logs](#Get-list-of-trend-logs) <br>
- [Get List of Systems and Subordinates](#Get-List-of-Systems-and-Subordinates) <br>
- [Example Workflows](#Example-Workflows) <br> 
 - [Workflow 1](#Workflow-1) <br>
 - [Workflow 2](#Workflow-2) <br>

# Reference doucments
****
Documents referenced to construct this document
<br/><br/>
1: [KbA0045: API to Pull Data From Kaizen](https://support.coppertreeanalytics.com/knowledge-base/kba/knowledge-base-articles/kba0045-api-to-pull-data-from-kaizen/)
<br/>
2: [Kaizen Database Upgrades](https://support.coppertreeanalytics.com/news-and-announcements/)

# Notebook setup
***

### Import packages

In [ ]:
# Imports from standard libraries
import requests
import json
import math
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import warnings

# Import API puller from supplementary file
from API_puller import API_puller

### Supplementary functions

In [ ]:
def check_response(r):
    '''Checks to ensure the expected response is received
    
    The accepted response from the API from the API is response [200] this
    function outputs raises an error if any other response is recived.
    '''
    if r.status_code == 200:
        return None
    else:
        raise ImportError('Received: [{}], Expected: [<Response [200]>]'.format(r))

In [ ]:
def print_n_lines(json_var, n_lines=20, indent=2):
    '''Pretty prints n lines of json file.
    
    This is used to make the outputs more compact
    '''
    pretty_str = str(json.dumps(json_var, indent=indent))
    length = len(pretty_str.splitlines())
    
    print('First {} / {} lines of json file:\n'.format(n_lines, length))
    for line in pretty_str.splitlines()[:n_lines]:
        print(line)
    print('..............')

In [ ]:
def assign_variables(example_info, specific_use):
    """Assigns and prints out building information and trend logs used as examples in this document
    """
    
    variables = example_info[specific_use]
    
    outputs = []
    for variable in variables:
        print(f'{variable} = {variables[variable][0]} # {variables[variable][1]}')
        outputs.append(variables[variable][0])
    
    return outputs  

### Load supplementary files

This step depends on two text files. (1) the required login information, and (2) the building numbers and example points included. They have been intentionally not been uploaded to GitHub - if you require this file contact Connor Brackley (connor.brackley@mail.concordia.ca)

In [ ]:
# Load text file
with open('login_info.txt') as f:
    login_info = json.loads(f.read())

# Assign variables
api_key = login_info['api_key']
client_id = login_info['api_key'] # The client ID is the same as the API key
client_secret = login_info['client_secret']
print('Login info successfully downloaded')

In [ ]:
with open('example_info.txt') as f:
    example_info = json.loads(f.read())
print('Examples have been successfully downloaded')

# Public Trend Log API
***

Excerpt from Kaizen Documentation:
><h3 id="To_get_trend_data:"> To get trend data: </h3>
><pre>https://kaizen.coppertreeanalytics.com/public_api/api/get_tl_data_start_end?api_key=&lt;api_key&gt;&amp;tl=&lt;tl_ref&gt;&amp;start=&lt;start_date&gt;&amp;end=&lt;end_date&gt;&amp;format=json&amp;data=&lt;raw or align&gt;
></pre>
>where: <ul>
><li> <strong>api_key</strong> is the user's Kaizen Key. Your API key can be found on the User Profile page.
></li> <li> <strong>tl_ref</strong> is the full reference for the trend log, in the format <building_id>.<device_id>.TL<instance_id> <ul>
><li> Example: 123.TL45 from the following building: <a class="natExternalLink" href="https://kaizen.coppertreeanalytics.com/v3/#/clients/551/buildings/2914" target="_blank" rel="noopener noreferrer">https://kaizen.coppertreeanalytics.com/v3/#/clients/551/buildings/2914</a> would be 2914.123.TL45. This will also be displayed in the URL when viewing this TL in the Vault.
></li></ul>
></instance_id></device_id></building_id></li> <li> <strong>start_date</strong> and <strong>end_date</strong> is the desired period start/end dates in the format YYYY-MM-DDThh:mm:ss e.g. 2018-01-01T00:00:00
></li> <li> <strong>&amp;data</strong> will determine if data's timestamps should be normalized or left in raw format <ul>
><li> <strong>raw</strong> will give the time-stamps and data exactly as they are read from the BAS
></li> <li> <strong>align</strong> will normalize the time-stamps to be aligned to 5-minute intervals. This must be defined to replicate the behavior of the BP API for point data.
></li></ul>
></li></ul>
><p></p>
><h4 id="Curl_Example:"> Curl Example: </h4>
><pre>curl “https://kaizen.coppertreeanalytics.com/public_api/api/get_tl_data_start_end?api_key=&lt;api_key&gt;&amp;tl=&lt;tl_ref&gt;&amp;start=&lt;start_date&gt;&amp;end=&lt;end_date&gt;&amp;format=json&amp;data=raw”
></pre>
><p></p>

### Python Implamentation:

In [ ]:
# Collect Example inputs
tl_ref, start_date, end_date = assign_variables(example_info, 'public_api_example')

In [ ]:
# Optional Inputs
format_output = 'json'
data_normalize = 'raw'

# Generate URL
url = 'https://kaizen.coppertreeanalytics.com/public_api/api/get_tl_data_start_end?' \
      'api_key={}&tl={}&start={}&end={}&format={}&data={}'.format(
            api_key, tl_ref, start_date, end_date, format_output, data_normalize)

# Download data from public API
r = requests.get(url)
check_response(r)

print_n_lines(r.json(), n_lines=20)

# Batch Download Public Trend Log API
***

Currently there is not built in method to batch load trend logs, however the function included here takes advantage of the public trend log API to download and organize a list of trend logs.

**API_puller documentation:**

    Retrieves data from Coppertree Analytics Kaizen API and organizes it into a single dataframe

    This function utilizes multithreading for to increase speed of API processing

    Parameters
    ----------
    trend_log_list : Pandas Dataframe
        a two column pandas dataframe with the trend log controller number in the in the first column
        and the name of the trend log in the second column

    API_key: str
         Your api key, which can be accessed through you're Kaizen account

    date_range: list, Format: ['YYYY-MM-DD', 'YYYY-MM-DD']
          a list of two date strings indicating start date and end date.
          Note: The date range is non inclusive, so the "end date" is not included in the API call

    resample: int, optional (default = none)
        Resample dataframe in minutes. For example to resample every 1 hour, enter resample=60.
        If none is received, no resampling will occur (warning: this may result in large outputs if
        event based sensors are included in query)

    Returns
    -------
    Dataframe
        Dataframe of the requested sensor inputs

### Python Implamentation:

In [ ]:
# Collect Example inputs
[trend_logs, labels] = assign_variables(example_info, 'batch_download')

In [ ]:
# Convert trend log lists into dataframe
trend_log_list = pd.DataFrame(list(zip(trend_logs, labels)), columns =['_id', 'object_name'])

# Employ API puller
df = API_puller(
    trend_log_list=trend_log_list,
    API_key=api_key,
    date_range=['2019-01-01', '2020-01-01'],
    resample=15
)

# Output resulting dataframe
df

# Access to other API's via the JWT Token
***

***Before accessing the other APIs, you need to get a JWT token. This acts as an added layer of security***

Excerpt from Kaizen Documentation:
><h2 id="Get_JWT_Token"> Get JWT Token </h2>
><p></p>
>First, the identity of the user must be confirmed to ensure that site data is going into the right hands. This is done through the production of an Auth0 JWT token.
><p></p>
>Reach out to <a href="mailto:customersolutions@coppertreeanalytics.com">customersolutions@coppertreeanalytics.com</a> and mention that you would like to work with the Insights API. You will receive a Client_ID and Client_Secret; use these parameters with the following cURL command to produce the token:
><pre>curl --request POST \ 
>     --url https://login-global.coppertreeanalytics.com/oauth/token
>     --header "content-type: application/x-www-form-urlencoded"
>     --data "grant_type=client_credentials"
>     --data "client_id=&lt;client_id&gt;"
>     --data "client_secret=&lt;client_secret&gt;"
>     --data "audience=organize"
></pre>
><p></p>
>Once a user acquires a token, they can use it to pull data
><p></p>

### Python Implamentation:

In [ ]:
url = 'https://login-global.coppertreeanalytics.com/oauth/token'

my_header = {'content-type': 'application/x-www-form-urlencoded'}
my_data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'audience': 'organize'
}

r = requests.post(url, headers=my_header, data=my_data)
check_response(r)
access_token = r.json()['access_token']
jwt_header = {'Authorization': 'Bearer ' + access_token}
print('Access token has been obtained')

# Get list of trend logs
***

Excerpt from Kaizen Documentation:
><h3 id="To_get_a_list_of_trend_logs_for_a_building:"> To get a list of trend logs for a building: </h3>
><pre>https://kaizen.coppertreeanalytics.com/api/v3/trend_log_objects/?building={};
></pre>
>where: <ul>
><li> <building_id> is the id for the building.
></building_id></li> <li> Make sure to add an authorization header:
></li> <li> "Authorization token <api_key>" where <api_key> is the user's Kaizen Key
></api_key></api_key></li> <li> Pagination: <ul>
><li> Add "&amp;page={page_number}" where page_number is the page you want to view.
><li> Add "&amp;page_size={page size}" where page size is the number of sensors that apear per page.
></page_number></li></ul>
></li></ul>
><p></p>
><h4 id="Curl_Example:_AN1"> Curl Example: </h4>
><pre>curl -H "Authorization: token &lt;api_key&gt;” “https://kaizen.coppertreeanalytics.com/api/v3/trend_log_objects/?building=&lt;building_id&gt;&amp;page=1”
></pre>

### Update:
>replace:
><pre>https://kaizen.coppertreeanalytics.com/api/v3/trend_log_objects/?building={}</pre>
>with
><pre>https://kaizen.coppertreeanalytics.com/yana/mongo/objects/?building={}&object_type=TL&min_device_index=1</pre>
>*Note: records with min_device_index of 0 (less than one) are built in kaizen analytics*

### Python Implamentation

In [ ]:
# Collect Example inputs
[building_number] = assign_variables(example_info, 'trend_log_list')

In [ ]:
# Set up URL
url = 'https://kaizen.coppertreeanalytics.com/yana/mongo/objects/?building={}&object_type=TL&min_device_index=1'.format(
    building_number
)

# Perform API Call
r = requests.get(url, headers=jwt_header)
check_response(r)

# Check response and print results
print_n_lines(r.json(), n_lines=50)

<font color=red> ***Note: not all trend logs were returned in a single call, you must set page size accordingly (see work flow 1 as an example)*** </font>

# Get List of Systems and Subordinates
***

Excerpt from Kaizen Documentation:
><h3 id="Get_data_from_the_Systems_feature"> Get data from the Systems feature </h3>
><p></p>
>To get a list of all Systems you can use this resource:<br>
><a class="natExternalLink" href="https://kaizen.coppertreeanalytics.com/api/v3/systems/?building=[building_id]" target="_blank" rel="noopener noreferrer">https://kaizen.coppertreeanalytics.com/api/v3/systems/?building=[building_id]</a> where the Building_ID can be found in the URL of any page in the Building.<br>
><strong>This endpoint uses pagination. Each call returns 500 results.</strong>
><p></p>
><h3 id="Example_API_call"> Example API call </h3>
><p></p>
>cURL Example to get a list of all Systems:
><pre>curl --request GET \
>     --url https://kaizen.coppertreeanalytics.com/api/v3/systems/?building=2156 \
>     --header "Authorization: Bearer &lt;jwt access_token&gt;"
></pre>

### Update:
>replace:
><pre>https://kaizen.coppertreeanalytics.com/api/v3/systems/?building=2156</pre>
>with
><pre>https://kaizen.coppertreeanalytics.com/yana/mongo/systems/?building=2156</pre>

### Python Implamentation:

In [ ]:
# Collect Example inputs
[building_number] = assign_variables(example_info, 'system_list')

In [ ]:
url = 'https://kaizen.coppertreeanalytics.com/yana/mongo/systems/?building={}'.format(
    building_number
)

r = requests.get(url, headers=jwt_header)
check_response(r)

# Print first n lines of output
print_n_lines(r.json(), n_lines=50)

<font color=red> ***Note: not all systems were returned in a single call, to get full list you must cycle through pages (see work flow 2 as an example)*** </font>

Other available options related to system searches <font color=red>(warning: this information may now be outdated)<font>
><pre>
>GET https://kaizen.coppertreeanalytics.com/api/v3/systems/?building={bldg}
># Get systems of a building with default page_size=10
># Default sort, device_index ascending and object_type ascending
>
>GET https://kaizen.coppertreeanalytics.com/api/v3/systems/?building={bldg}&search=valve
># Get systems of building which 'Tags', 'Subordinate_Tags', 'Subordinate_List' has "valve"
>
>GET https://kaizen.coppertreeanalytics.com/api/v3/systems/?building={bldg}&ordering=-device_index
># Get systems of a building with device_index descending order
>
>GET https://kaizen.coppertreeanalytics.com/api/v3/systems/?building={bldg}&device_index=0
># Get systems whose device_index is 0
>
>GET https://kaizen.coppertreeanalytics.com/api/v3/systems/?building={bldg}&page_size=20
># Get systems of a building with page_size=20
></pre>

# Example Workflows
___

### Workflow 1

Use api to list your needed trend logs. This is done by getting a list of all trend logs and then filtering as needed. This is particularly useful when you need to get a long list of trend logs, where Kaizens build in search functionality can be particularly limiting.

In [ ]:
# Collect Example inputs
[building_number, controller_min_max, keyword_list] = assign_variables(example_info, 'example_1')

In [ ]:
# Initial API query gets sensor count
url = 'https://kaizen.coppertreeanalytics.com/yana/mongo/objects/?building={}&object_type=TL&min_device_index=1&page_size=1'.format(building_number)
r = requests.get(url, headers=jwt_header)
count = r.json()['meta']['pagination']['count']

# Second API query gets full sensor list
url = 'https://kaizen.coppertreeanalytics.com/yana/mongo/objects/?building={}&object_type=TL&min_device_index=1&page_size={}'.format(building_number, count)
r = requests.get(url, headers=jwt_header)

# Convert to pandas dataframe
df = pd.DataFrame.from_dict(r.json()['results'])[['_id', 'Object_Name']]

In [ ]:
def filter_tl_list(df, search_items, method='controller'):
    """This function filters a TL list based on the search item and method.
    
    Parameters
    ----------
    df : Pandas Dataframe
         Dataframe with all building sensors
         
    Search items: list
         List of seach items as strings.
    
    Search items: 'keyword', 'controller', 'trend_log', default 'controller'
          keyword searches object names for text string
          Controler searches controller number
          Trend log searchers 

    Raises
    ------
        If any of the search criteria's produces no logs this function will produce a warning
        
        If the search criteria results in duplicate's this function will remove them and 
        produce a warning
        
    
    Returns
    -------
    df_filtered, Dataframe
        Filtered dataframe based on input parameters
    
    """
    
    # Check Input Criteria
    if type(search_items) is not list:
        raise TypeError('search_item should be a list')
    if method not in ['keyword', 'controller', 'trend_log']:
        raise ValueError('search method input not recognized')

    # Convert inputs to str to allow integers as inputs 
    search_items = map(str, search_items)
    
    # Initialize dataframe
    df_filtered = pd.DataFrame()
    
    # Loop through search items and filter based on search method
    for search_item in search_items:
        if method == 'keyword':
            df_search = df[df['Object_Name'].str.contains(search_item)]
        elif method == 'controller':
            df_search = df[df['_id'].str.contains('.'+search_item+'.')]
        elif method == 'trend_log':
            df_search = df[df['_id'].str.contains('.TL'+'.'+search_item)]

        # If search produces no logs, output a warning warning, if results found not append to dataframe
        if df_search.empty:
            warn_txt = 'search item [{}] yeilded no results'.format(search_item)
            warnings.warn(warn_txt)
        else:
            df_filtered = df_filtered.append(df_search)
    
    # Count duplicates and warn user if any are found
    duplicate_count = df_filtered.duplicated(subset='_id', keep='first').sum()
    if duplicate_count != 0:
            warn_txt = '{} duplicate values were found and removed, consider refining search keywords'.format(duplicate_count)
            warnings.warn(warn_txt) 
    
    # Drop duplicates and reset index on output
    df_filtered = df_filtered.drop_duplicates().reset_index(drop=True)
    return df_filtered

In [ ]:
#Example search: search first by contoller and then by keyword
controller_list = list(range(int(controller_min_max[0]), int(controller_min_max[1])))

df_filtered = filter_tl_list(df, controller_list, method='controller')
df_filtered = filter_tl_list(df_filtered, keyword_list, method='keyword')
df_filtered

In [ ]:
# The API_puller can be found in "API_puller.py"
# This function uses the Keizens public API to retrieve sensor data 
trend_logs = API_puller(
    trend_log_list=df_filtered,
    API_key=api_key,
    date_range=['2019-01-01', '2020-01-01'],
    resample=15
)

In [ ]:
# Plot data retrieved from the API
ax = trend_logs.plot(figsize=(15,8), title='Data downloaded from API', ylabel='Temperature (°C)', xlabel='Timescale')
ax.legend(bbox_to_anchor=(1.0, 1.0));

#### Ideas for future implementation:
 - Incorporate "Notify_Type" Parameter to differentiate handing of event based and time step based recordings

### Workflow 2
Use api to access systems and subordinates using tags.

In [ ]:
# Collect Example inputs
[building_number] = assign_variables(example_info, 'example_2')

In [ ]:
url = 'https://kaizen.coppertreeanalytics.com/yana/mongo/systems/?building={}&page_size=1'.format(
    building_number
)
r = requests.get(url, headers=jwt_header)
count = r.json()['meta']['pagination']['count']

url = 'https://kaizen.coppertreeanalytics.com/yana/mongo/systems/?building={}&page_size={}'.format(
    building_number, count
)
r = requests.get(url, headers=jwt_header)

In [ ]:
# Create dataframe
df = pd.json_normalize(r.json()['results'])
df = df.apply(lambda x: x.explode() if x.name in ['Subordinate_Annotations', 'Subordinate_List', 'Subordinate_Tags'] else x)

In [ ]:
search_term = 'Mixed Air Temperature'
df_filtered = df[df['Subordinate_Tags']==search_term]
df_filtered = df_filtered.drop_duplicates(subset='Subordinate_List', keep="first")
df_filtered = df_filtered[['Subordinate_List', 'Subordinate_Annotations']]
df_filtered['Subordinate_List'] = str(building_number) + '.' + df_filtered['Subordinate_List'].astype(str)
df_filtered

In [ ]:
df = API_puller(
    trend_log_list=df_filtered,
    API_key=api_key,
    date_range=['2019-01-01', '2020-01-01'],
    resample=15
)

In [ ]:
ax = df.plot(figsize=(15,8), title='Data downloaded from API',ylabel='°C', xlabel='Timescale')
ax.legend(bbox_to_anchor=(1.0, 1.0));

#### Ideas for future implementation:
 - Organize API calls based on system hierarchy